In [23]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (120 kB/s)
Reading package li

In [2]:
reviewsDf = spark.read.json('/content/sample_data/CDs_and_Vinyl_5.json')

In [24]:
reviewsDf.filter(reviewsDf.reviewTime.isNull()).show()

+---------------+----+-------+-------+----------+----------+----------+------------+-------+--------------+
|_corrupt_record|asin|helpful|overall|reviewText|reviewTime|reviewerID|reviewerName|summary|unixReviewTime|
+---------------+----+-------+-------+----------+----------+----------+------------+-------+--------------+
+---------------+----+-------+-------+----------+----------+----------+------------+-------+--------------+



In [4]:
reviewsDf.show()

+---------------+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|_corrupt_record|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+---------------+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|           null|0307141985|[14, 15]|    5.0|I don't know who ...| 10 6, 2005|A3IEV6R2B7VW5Z|         J. Anderson|LISTEN TO THE PUB...|    1128556800|
|           null|0307141985|  [2, 2]|    4.0|Thanksgiving is d...|11 23, 2011|A2H3ISQ4QB95XN|       Joseph Brando|Rankin/Bass Does ...|    1322006400|
|           null|0307141985|[38, 38]|    5.0|This is a Thanksg...|07 14, 2003| A6GMEO3VRY51S|            microjoe|Thanksgiving Holi...|    1058140800|
|           null|0307141985|[15, 16]|    5.0|This is the BEST ...| 11 6, 2003|A3E102F6LPUF1J|R

In [29]:
from pyspark.sql.functions import *
df1 = reviewsDf.withColumn("reviewTime",regexp_replace("reviewTime"," ","-")).withColumn("reviewTime",regexp_replace("reviewTime",",",""))

In [30]:
df1.show()

+---------------+----------+--------+-------+--------------------+----------+--------------+--------------------+--------------------+--------------+
|_corrupt_record|      asin| helpful|overall|          reviewText|reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+---------------+----------+--------+-------+--------------------+----------+--------------+--------------------+--------------------+--------------+
|           null|0307141985|[14, 15]|    5.0|I don't know who ...| 10-6-2005|A3IEV6R2B7VW5Z|         J. Anderson|LISTEN TO THE PUB...|    1128556800|
|           null|0307141985|  [2, 2]|    4.0|Thanksgiving is d...|11-23-2011|A2H3ISQ4QB95XN|       Joseph Brando|Rankin/Bass Does ...|    1322006400|
|           null|0307141985|[38, 38]|    5.0|This is a Thanksg...|07-14-2003| A6GMEO3VRY51S|            microjoe|Thanksgiving Holi...|    1058140800|
|           null|0307141985|[15, 16]|    5.0|This is the BEST ...| 11-6-2003|A3E102F6LPUF1J|Richard 

In [20]:
from pyspark.sql.functions import col,udf
from datetime import datetime
from pyspark.sql.types import DateType


In [45]:
df2=df1.select(col("asin"),col("helpful"),col("overall"),col("reviewText"),to_date(col("reviewTime"),"MM-dd-yyyy").alias("reviewTime"),col("reviewerID"),col("reviewerName"),col("summary"),col("unixReviewTime"))

In [46]:
df2.show()

+----------+--------+-------+--------------------+----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText|reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+----------+--------------+--------------------+--------------------+--------------+
|0307141985|[14, 15]|    5.0|I don't know who ...|2005-10-06|A3IEV6R2B7VW5Z|         J. Anderson|LISTEN TO THE PUB...|    1128556800|
|0307141985|  [2, 2]|    4.0|Thanksgiving is d...|2011-11-23|A2H3ISQ4QB95XN|       Joseph Brando|Rankin/Bass Does ...|    1322006400|
|0307141985|[38, 38]|    5.0|This is a Thanksg...|2003-07-14| A6GMEO3VRY51S|            microjoe|Thanksgiving Holi...|    1058140800|
|0307141985|[15, 16]|    5.0|This is the BEST ...|2003-11-06|A3E102F6LPUF1J|Richard J. Goldsc...|BEST THANKSGIVING...|    1068076800|
|0307141985|[11, 12]|    5.0|It's been a numbe...|2006-03-01|A

In [47]:
df2.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: date (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [ ]:
df2.to_sql('Review_table', engine, if_exists='replace')

In [48]:
df2.write.parquet("/content/sample_data/Reviews.parquet") 